In [66]:
import json
import os

from collections import Counter
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA


In [65]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Chankoo\AppData\Roaming\nltk_data...


True

In [2]:
os.getcwd()

'C:\\Users\\Chankoo\\Desktop\\GitHub\\BOAZ-projects\\airbnb-NLP'

In [3]:
os.chdir('./../airbnb-data')

In [4]:
with open('pos_review.json') as fp:
    review = json.load(fp)

In [5]:
len(review)

30569

In [48]:
with open('voca_freq_dic.json') as fp:
    voca_freq_dic = json.load(fp)

In [49]:
len(voca_freq_dic)

147119

In [50]:
tmp = {0: [('be_V', 0.14537306),
  ('room_N', 0.027824974),
  ('have_V', 0.023344692),
  ('not_R', 0.02325792),
  ('little_J', 0.0128844995),
  ('small_J', 0.0116364965),
  ('bathroom_N', 0.011063971),
  ('too_R', 0.010339574),
  ('bed_N', 0.0100401165),
  ('use_V', 0.010013665),
  ('floor_N', 0.00997476),
  ('so_R', 0.009677156),
  ('people_N', 0.008907996),
  ('bit_N', 0.008873436),
  ('space_N', 0.008067901),
  ('water_N', 0.007713604),
  ('thing_N', 0.0075807776),
  ('big_J', 0.007515625),
  ('kitchen_N', 0.0070377397),
  ('facility_N', 0.007004679),
  ('well_R', 0.006998033),
  ('towel_N', 0.006157327),
  ('also_R', 0.006134526),
  ('far_R', 0.005750602),
  ('machine_N', 0.005465284),
  ('sleep_V', 0.005457032),
  ('enough_R', 0.005420316),
  ('access_N', 0.005417895),
  ('available_J', 0.005136294),
  ('problem_N', 0.0050424635)],
 1: [('be_V', 0.103530966),
  ('station_N', 0.061354417),
  ('subway_N', 0.032991868),
  ('very_R', 0.030921662),
  ('location_N', 0.026547782),
  ('restaurant_N', 0.024023393),
  ('convenient_J', 0.023671063),
  ('walk_V', 0.021608574),
  ('minute_N', 0.019378979),
  ('close_R', 0.017736545),
  ('locate_V', 0.016287815),
  ('bus_N', 0.01621343),
  ('area_N', 0.015900152),
  ('store_N', 0.015561452),
  ('easy_J', 0.01511124),
  ('just_R', 0.014580455),
  ('away_R', 0.014054836),
  ('also_R', 0.013332136),
  ('many_J', 0.012269877),
  ('walk_N', 0.012204578),
  ("it's_N", 0.011717381),
  ('convenience_N', 0.011196179),
  ('close_J', 0.0106944535),
  ('landlord_N', 0.010456182),
  ('get_V', 0.010118616),
  ('apartment_N', 0.009564664),
  ('take_V', 0.009525702),
  ('street_N', 0.009133658),
  ('hongdae_N', 0.008938996),
  ('min_N', 0.00806987)],
 2: [('be_V', 0.1839694),
  ('very_R', 0.06898027),
  ('clean_J', 0.066435896),
  ('nice_J', 0.061047398),
  ('room_N', 0.04001629),
  ('comfortable_J', 0.03529435),
  ('place_N', 0.033623092),
  ('house_N', 0.033101045),
  ('apartment_N', 0.025324035),
  ("it's_N", 0.02145301),
  ('quiet_J', 0.017014496),
  ('convenient_N', 0.015896525),
  ('really_R', 0.015816757),
  ('airport_N', 0.015232947),
  ('so_R', 0.010995699),
  ('look_V', 0.010577824),
  ('picture_N', 0.010254646),
  ('well_R', 0.009996955),
  ('spacious_J', 0.008540962),
  ('neighborhood_N', 0.008289231),
  ("it's_J", 0.007964953),
  ('super_J', 0.0078181885),
  ('quite_R', 0.0077534383),
  ('cozy_J', 0.0073792483),
  ('beautiful_J', 0.0066576796),
  ('exactly_R', 0.006564339),
  ('view_N', 0.006304495),
  ('cozy_N', 0.0062290356),
  ('accommodation_N', 0.0061460896),
  ('excellent_J', 0.0060593183)],
 3: [('good_J', 0.16281429),
  ('be_V', 0.11312827),
  ('very_R', 0.053128365),
  ('helpful_J', 0.05195052),
  ('location_N', 0.05124764),
  ('well_R', 0.03354444),
  ('love_V', 0.02612817),
  ('as_R', 0.023941612),
  ('really_R', 0.022668261),
  ('respond_V', 0.015254475),
  ('clean_N', 0.014086787),
  ('nice_N', 0.013732725),
  ('communication_N', 0.012630968),
  ('take_V', 0.011255137),
  ('awesome_J', 0.009965467),
  ('have_V', 0.009748615),
  ('rest_N', 0.008570963),
  ('staff_N', 0.008565894),
  ('mr_N', 0.008015735),
  ('recommended_J', 0.007751195),
  ('english_N', 0.0070911297),
  ('so_R', 0.006903312),
  ('communicate_V', 0.0067509892),
  ('soon_R', 0.00601351),
  ('palace_N', 0.0059772106),
  ('speak_V', 0.0058694137),
  ('perfect_J', 0.005677414),
  ("it's_N", 0.005628891),
  ('enthusiastic_J', 0.0054090726),
  ('talk_V', 0.0053982786)],
 4: [('stay_N', 0.037552293),
  ('make_V', 0.029919602),
  ('provide_V', 0.02769604),
  ('have_V', 0.027451187),
  ('home_N', 0.027110588),
  ('get_V', 0.022025237),
  ('even_R', 0.021529136),
  ('out_R', 0.02025835),
  ('up_R', 0.01865333),
  ('find_V', 0.01728073),
  ('check_V', 0.01590451),
  ('do_V', 0.015456957),
  ('feel_V', 0.015186817),
  ('apartment_N', 0.013967136),
  ('shop_N', 0.013669611),
  ('enjoy_V', 0.013669383),
  ('wifi_N', 0.013366466),
  ('wonderful_J', 0.013281185),
  ('give_V', 0.013029154),
  ('be_V', 0.012245869),
  ('few_J', 0.0121913925),
  ('luggage_N', 0.011880643),
  ('arrive_V', 0.0117918365),
  ('really_R', 0.011740341),
  ('day_N', 0.011445442),
  ('also_R', 0.010479523),
  ('check_N', 0.010396812),
  ('leave_V', 0.009931349),
  ('night_N', 0.009691304),
  ('not_R', 0.009553752)],
 5: [('place_N', 0.1464667),
  ('great_J', 0.10717892),
  ('recommend_V', 0.059553694),
  ('be_V', 0.05873483),
  ('stay_V', 0.036092173),
  ('experience_N', 0.027138183),
  ('location_N', 0.025593793),
  ('korea_N', 0.024924552),
  ('stay_N', 0.023986349),
  ('highly_R', 0.021492453),
  ('great_N', 0.018332617),
  ('apartment_N', 0.015134046),
  ('really_R', 0.013760422),
  ('responsive_J', 0.011146173),
  ('amazing_J', 0.0099802455),
  ('definitely_R', 0.009918727),
  ('safe_J', 0.008971682),
  ('market_N', 0.00810238),
  ('anyone_N', 0.0076547945),
  ('main_J', 0.007577685),
  ('enjoy_V', 0.007540161),
  ('such_J', 0.00714247),
  ('need_N', 0.006872107),
  ('book_N', 0.006809102),
  ('near_J', 0.0061891577),
  ('lovely_J', 0.006051446),
  ('neighbourhood_N', 0.00585503),
  ('flat_J', 0.0057031875),
  ('traveler_N', 0.005486759),
  ('perfect_N', 0.0054737623)],
 6: [('seoul_N', 0.10629787),
  ('stay_V', 0.07450916),
  ('again_R', 0.07379789),
  ('here_R', 0.052844416),
  ('time_N', 0.047839932),
  ('come_V', 0.04599049),
  ('next_J', 0.041201647),
  ('go_V', 0.037889875),
  ('back_R', 0.033760805),
  ('definitely_R', 0.032483123),
  ('want_V', 0.030077608),
  ('there_R', 0.02870401),
  ('visit_V', 0.027000032),
  ('shopping_N', 0.0126639195),
  ("i'm_N", 0.012112648),
  ('short_J', 0.01123441),
  ('fantastic_J', 0.008616221),
  ('central_J', 0.007990689),
  ('spot_N', 0.0076173455),
  ('shop_V', 0.0071637966),
  ('accessible_J', 0.0070721614),
  ("i'll_N", 0.0068555763),
  ('trip_N', 0.0066671693),
  ('visit_N', 0.0065609585),
  ('use_V', 0.0065015047),
  ('see_V', 0.0062890495),
  ('book_V', 0.006235881),
  ('wait_V', 0.005876947),
  ('return_V', 0.0052519836),
  ('travel_V', 0.005102341)],
 7: [('have_V', 0.07795763),
  ('be_V', 0.06771594),
  ('everything_N', 0.033561654),
  ('need_V', 0.03292147),
  ('time_N', 0.029921964),
  ('so_R', 0.024076074),
  ('night_N', 0.02294283),
  ('day_N', 0.022272876),
  ('best_J', 0.020033248),
  ('stay_V', 0.019138478),
  ('house_N', 0.018787306),
  ('first_J', 0.016388895),
  ('family_N', 0.016274076),
  ('airbnb_N', 0.015712097),
  ('trip_N', 0.015101894),
  ('much_J', 0.013730331),
  ('live_V', 0.012128284),
  ('friend_N', 0.011398903),
  ('get_V', 0.010528335),
  ('like_V', 0.010181942),
  ('amenity_N', 0.009319371),
  ('make_V', 0.009051389),
  ('go_V', 0.00887664),
  ('meet_V', 0.008233865),
  ('perfect_J', 0.008026695),
  ('guest_N', 0.00791008),
  ('ever_R', 0.00723431),
  ('choose_V', 0.0070612817),
  ('say_V', 0.006622531),
  ("don't_V", 0.006431209)],
 8: [('help_V', 0.053845465),
  ('breakfast_N', 0.020658312),
  ('korean_J', 0.020005353),
  ('owner_N', 0.018348051),
  ('food_N', 0.01608563),
  ('lot_N', 0.015571722),
  ('eat_V', 0.01546228),
  ('morning_N', 0.015013883),
  ('guesthouse_N', 0.013262614),
  ('egg_N', 0.011632346),
  ('also_R', 0.011181566),
  ('free_J', 0.011172976),
  ('even_R', 0.011058747),
  ('cook_V', 0.0105845705),
  ('message_N', 0.010321902),
  ('taxi_N', 0.010276527),
  ('need_V', 0.009034017),
  ('delicious_J', 0.009027167),
  ('anything_N', 0.008953328),
  ('tell_V', 0.008533005),
  ('value_N', 0.008457952),
  ("there's_V", 0.008409349),
  ('prepare_V', 0.008024097),
  ('cosy_J', 0.007933274),
  ('traditional_J', 0.007916343),
  ('late_R', 0.00692333),
  ('try_V', 0.0067711715),
  ('fun_N', 0.006658744),
  ('hostel_N', 0.0063502034),
  ('winter_N', 0.0060010254)],
 9: [('be_V', 0.13986205),
  ('host_N', 0.103992924),
  ('very_R', 0.08209433),
  ('kind_N', 0.03637853),
  ('thank_N', 0.029831955),
  ('friendly_J', 0.026155282),
  ('question_N', 0.017913863),
  ('always_R', 0.017277787),
  ('thanks_N', 0.015141893),
  ('so_R', 0.01355928),
  ('overall_J', 0.012453678),
  ('quickly_R', 0.009686644),
  ('quick_J', 0.009576056),
  ('answer_V', 0.009474822),
  ('also_R', 0.009097035),
  ('i_N', 0.008681364),
  ('much_R', 0.008679146),
  ('give_V', 0.008490807),
  ('instruction_N', 0.008098223),
  ('information_N', 0.007977161),
  ('really_R', 0.0070621003),
  ('air_N', 0.0069595543),
  ('welcome_V', 0.006732293),
  ('take_V', 0.0062996983),
  ('reply_V', 0.0062171645),
  ('hospitality_N', 0.0057337605),
  ('response_N', 0.005612963),
  ('care_N', 0.005292849),
  ('park_N', 0.0052119857),
  ('warm_J', 0.0045089344)]}

In [61]:
aspect_keywords = []
for i,k_words in tmp.items():
    keys,b = zip(*k_words)
    aspect_keywords.append(keys)


In [62]:
aspect_keywords

[('be_V',
  'room_N',
  'have_V',
  'not_R',
  'little_J',
  'small_J',
  'bathroom_N',
  'too_R',
  'bed_N',
  'use_V',
  'floor_N',
  'so_R',
  'people_N',
  'bit_N',
  'space_N',
  'water_N',
  'thing_N',
  'big_J',
  'kitchen_N',
  'facility_N',
  'well_R',
  'towel_N',
  'also_R',
  'far_R',
  'machine_N',
  'sleep_V',
  'enough_R',
  'access_N',
  'available_J',
  'problem_N'),
 ('be_V',
  'station_N',
  'subway_N',
  'very_R',
  'location_N',
  'restaurant_N',
  'convenient_J',
  'walk_V',
  'minute_N',
  'close_R',
  'locate_V',
  'bus_N',
  'area_N',
  'store_N',
  'easy_J',
  'just_R',
  'away_R',
  'also_R',
  'many_J',
  'walk_N',
  "it's_N",
  'convenience_N',
  'close_J',
  'landlord_N',
  'get_V',
  'apartment_N',
  'take_V',
  'street_N',
  'hongdae_N',
  'min_N'),
 ('be_V',
  'very_R',
  'clean_J',
  'nice_J',
  'room_N',
  'comfortable_J',
  'place_N',
  'house_N',
  'apartment_N',
  "it's_N",
  'quiet_J',
  'convenient_N',
  'really_R',
  'airport_N',
  'so_R',
  'lo

In [73]:
def get_aspect_terms(aspects, vocab_dict):
    aspect_terms = []
    w_notfound = []
    
    for aspect_kws in aspects:
        aspect = []
        for w in aspect_kws:
            if w in vocab_dict:
                aspect.append(w)
            else:
                w_notfound.append(w)
        aspect_terms.append(aspect)
    #We are only using one hotel review file, as we keep inceasing the number of files words not found will decrease.
    # print "Words not found in vocab:", ' '.join(w_notfound)
    return aspect_terms

In [76]:
get_aspect_terms(aspect_keywords,voca_freq_dic)

In [75]:
def chi_sq(a,b,c,d):
    c1 = a
    c2 = b - a
    c3 = c - a
    c4 = d - b - c + a
    nc =  d
    return nc * (c1*c4 - c2*c3) * (c1*c4 - c2*c3)/((c1+c3) * (c2+c4) * (c1+c2) * (c3+c4))

In [70]:
def chi_sq_mat():
    global aspect_words, aspect_sent, num_words
    asp_rank = np.zeros(aspect_words.shape)
    for i in range(len(aspect_terms)):
        for j in range(len(vocab)):
            asp_rank[i][j] = chi_sq(aspect_words[i][j], num_words[j], aspect_sent[i], len(sent))
    return asp_rank

In [72]:
def aspect_segmentaion():
    #Sentiment analysis
    sid = SIA()

    #INPUT
    #review, this algo needs all the review. Please process dataset.
    file="Data/Texts/hotel_72572_parsed.txt"
    reviews, all_ratings = load_file(file)

    #selection threshold
    p = 5
    #Iterations 
    # I = 10
    I = 1

    #Create Vocabulary
    review_sent, review_actual, only_sent = parse_to_sentence(reviews)
    vocab, vocab_dict = create_vocab(only_sent)

    #Aspect Keywords
    aspect_file = "aspect_keywords.csv"
    aspect_terms = get_aspect_terms(aspect_file, vocab_dict)

    label_text = ['Value', 'Rooms', 'Location', 'Cleanliness', 'Check in/Front Desk', 'Service', 'Business Service']
    # print aspect_terms

    #ALGORITHM
    review_labels = []
    k = len(aspect_terms)
    v = len(vocab)
    aspect_words = np.zeros((k,v))
    aspect_sent = np.zeros(k)
    num_words = np.zeros(v)

    for i in range(I):
        for r in review_sent:
            labels = []
            for s in r:
                count = np.zeros(len(aspect_terms))
                i = 0
                for a in aspect_terms:
                    for w in s:
                        if vocab_dict.has_key(w):
                            num_words[vocab_dict[w]] += 1
                            if w in a:
                                count[i] += 1
                    i = i + 1
                if max(count) > 0:
                    la = np.where(np.max(count) == count)[0].tolist()
                    labels.append(la)
                    for i in la:
                        aspect_sent[i] += 1
                        for w in s:
                            if vocab_dict.has_key(w):
                                aspect_words[i][vocab_dict[w]] += 1
                else:
                    labels.append([])
            review_labels.append(labels)
            # aspect_w_rank = chi_sq_mat()
            # new_labels = []
            # for na in aspect_w_rank:
            # 	x = np.argsort(na)[::-1][:p]
            # 	new_labels.append(x)
                # for k,v in vocab_dict.items():
                # 	if vocab_dict[k] in x:
                # 		print k
                # print 
            # sys.exit()


    ratings_sentiment = []
    for r in review_actual:
        sentiment = []
        #aspect ratings based on sentiment
        for s in r:
            ss = sid.polarity_scores(s)
            sentiment.append(ss['compound'])
        ratings_sentiment.append(sentiment)

    #Aspect Ratings Per Review
    aspect_ratings = []
    for i,r in enumerate(review_labels):
        rating = np.zeros(7)
        count = np.zeros(7)
        rs = ratings_sentiment[i] 
        for j,l in enumerate(r):
            for k in range(7):
                if k in l:
                    rating[k] += rs[j]
            for k in range(7):
                if count[k] != 0:
                    rating[k] /= count[k]
        #Map from -[-1,1] to [1,5]
        for k in range(7):
            if rating[k] != 0:
                rating[k] = int(round((rating[k]+1)*5/2))
        aspect_ratings.append(rating)
    return aspect_ratings, all_ratings

    # n = 0
    # print review_actual[n], '\n', review_labels[n]
    # print ratings_sentiment[n], '\n', aspect_ratings[n]
    # print len(all_ratings), len(reviews), all_ratings[0]
    # sys.exit()
    # return aspect_ratings

    # print sent[5:9], labels[5:9]
    # print zip(actual_sent, labels)[:10]
    # print zip(actual_sent, sentiment)[:10]